# **Kaggle – DataTops®**

Tu TA ha decidido cambiar de aires y, por eso, ha comprado una tienda de portátiles. Sin embargo, su única especialidad es Data Science, por lo que ha decidido crear un modelo de ML para establecer los mejores precios.

¿Podrías ayudar a tu profe a mejorar ese modelo?

## Aspectos importantes

- Última submission:
	- Mañana: 17 de febrero a las 5pm
	- Tarde: 19 de febrero a las 5pm
- **Enlace de la competición**: https://www.kaggle.com/t/c5cc87b50c4b4770bdc8f5acbe15577d
- **Requisito**: Estar registrado en [Kaggle](https://www.kaggle.com/)

## Métrica

El error cuadrático medio (RMSE, por sus siglas en inglés) es una medida de la desviación estándar de los residuos (errores de predicción). Los residuos representan la diferencia entre los valores observados y los valores predichos por el modelo. El RMSE indica qué tan dispersos están estos errores: cuanto menor es el RMSE, más cercanas están las predicciones a los valores reales. En otras palabras, el RMSE mide qué tan bien se ajusta la línea de regresión a los datos.

$$ RMSE = \sqrt{\frac{1}{n}\Sigma_{i=1}^{n}{\Big(\frac{d_i -f_i}{\sigma_i}\Big)^2}}$$

## 1. Librerías

In [44]:
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import urllib.request


pd.options.mode.copy_on_write = True

In [45]:
from google.colab import drive


# Mount the Google Drive at /content/drive
drive.mount("/content/drive")

# Verify by listing the files in the drive
# !ls /content/drive/My\ Drive/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 2. Datos

In [46]:
# Para que funcione necesitas bajarte los archivos de datos de Kaggle
path = "/content/drive/MyDrive/Colab Notebooks/The Bridge/data"
df = pd.read_csv(f"{path}/train.csv")
df.shape

(912, 13)

### 2.1 Exploración de los datos

In [47]:
def get_quality(dataframe: pd.DataFrame):
	if dataframe is None:
		raise ValueError("Dataframe sin especificar.")

	if not isinstance(dataframe, pd.DataFrame):
		raise ValueError("El dataframe debe ser pd.DataFrame")

	records = dataframe.shape[0]

	unique = dataframe.nunique().rename("Cardinality")
	cardinality = (unique/records).rename("Uniqueness")
	null = (dataframe.isna().sum()).rename("Null")

	data = pd.DataFrame(data=[unique, cardinality, null])
	return data.T

In [48]:
get_quality(df)

,Cardinality,Uniqueness,Null
laptop_ID,912.0,1.000000,0.0
Company,19.0,0.020833,0.0
Product,480.0,0.526316,0.0
TypeName,6.0,0.006579,0.0
Inches,17.0,0.018640,0.0
ScreenResolution,36.0,0.039474,0.0
Cpu,107.0,0.117325,0.0
Ram,9.0,0.009868,0.0
Memory,37.0,0.040570,0.0
Gpu,93.0,0.101974,0.0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 912 entries, 0 to 911
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   laptop_ID         912 non-null    int64  
 1   Company           912 non-null    object 
 2   Product           912 non-null    object 
 3   TypeName          912 non-null    object 
 4   Inches            912 non-null    float64
 5   ScreenResolution  912 non-null    object 
 6   Cpu               912 non-null    object 
 7   Ram               912 non-null    object 
 8   Memory            912 non-null    object 
 9   Gpu               912 non-null    object 
 10  OpSys             912 non-null    object 
 11  Weight            912 non-null    object 
 12  Price_in_euros    912 non-null    float64
dtypes: float64(2), int64(1), object(10)
memory usage: 92.8+ KB


In [50]:
df.head()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
0,755,HP,250 G6,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.86kg,539.00
1,618,Dell,Inspiron 7559,Gaming,15.6,Full HD 1920x1080,Intel Core i7 6700HQ 2.6GHz,16GB,1TB HDD,Nvidia GeForce GTX 960<U+039C>,Windows 10,2.59kg,879.01
2,909,HP,ProBook 450,Notebook,15.6,Full HD 1920x1080,Intel Core i7 7500U 2.7GHz,8GB,1TB HDD,Nvidia GeForce 930MX,Windows 10,2.04kg,900.00
3,2,Apple,Macbook Air,Ultrabook,13.3,1440x900,Intel Core i5 1.8GHz,8GB,128GB Flash Storage,Intel HD Graphics 6000,macOS,1.34kg,898.94
4,286,Dell,Inspiron 3567,Notebook,15.6,Full HD 1920x1080,Intel Core i3 6006U 2.0GHz,4GB,1TB HDD,AMD Radeon R5 M430,Linux,2.25kg,428.00


In [51]:
df.tail()

,laptop_ID,Company,Product,TypeName,Inches,ScreenResolution,Cpu,Ram,Memory,Gpu,OpSys,Weight,Price_in_euros
907,28,Dell,Inspiron 5570,Notebook,15.6,Full HD 1920x1080,Intel Core i5 8250U 1.6GHz,8GB,256GB SSD,AMD Radeon 530,Windows 10,2.2kg,800.00
908,1160,HP,Spectre Pro,2 in 1 Convertible,13.3,Full HD / Touchscreen 1920x1080,Intel Core i5 6300U 2.4GHz,8GB,256GB SSD,Intel HD Graphics 520,Windows 10,1.48kg,1629.00
909,78,Lenovo,IdeaPad 320-15IKBN,Notebook,15.6,Full HD 1920x1080,Intel Core i5 7200U 2.5GHz,8GB,2TB HDD,Intel HD Graphics 620,No OS,2.2kg,519.00
910,23,HP,255 G6,Notebook,15.6,1366x768,AMD E-Series E2-9000e 1.5GHz,4GB,500GB HDD,AMD Radeon R2,No OS,1.86kg,258.00
911,229,Dell,Alienware 17,Gaming,17.3,IPS Panel Full HD 1920x1080,Intel Core i7 7700HQ 2.8GHz,16GB,256GB SSD + 1TB HDD,Nvidia GeForce GTX 1060,Windows 10,4.42kg,2456.34


In [52]:
df.describe()

,laptop_ID,Inches,Price_in_euros
count,912.000000,912.000000,912.000000
mean,650.312500,14.981579,1111.724090
std,382.727748,1.436719,687.959172
min,2.000000,10.100000,174.000000
25%,324.750000,14.000000,589.000000
50%,636.500000,15.600000,978.000000
75%,982.250000,15.600000,1483.942500
max,1320.000000,18.400000,6099.000000


#### Transformaciones

##### `ScreenResolution`

In [53]:
import re


def extract_screen_data(value):
	touchscreen_match = re.search(r"Touchscreen", value, re.IGNORECASE)
	panel_match = re.search(r"(IPS Panel|LCD|OLED)", value, re.IGNORECASE)
	type_match = re.search(r"(Quad|Full|Ultra)\s?HD\+?", value, re.IGNORECASE)
	resolution_match = re.search(r"\d{3,4}x\d{3,4}", value, re.IGNORECASE)

	return {
		"has_touchscreen": True if touchscreen_match else False,
		"screen_panel": panel_match.group(0) if panel_match else None,
		"screen_type": type_match.group(0) if type_match else None,
		"screen_resolution": resolution_match.group(0) if resolution_match else None
	}

screen_data = df["ScreenResolution"].apply(extract_screen_data)
df = pd.concat([df, pd.DataFrame(screen_data.tolist())], axis=1)
df.sample(5).iloc[:, -5:]

,Price_in_euros,has_touchscreen,screen_panel,screen_type,screen_resolution
547,3499.0,False,None,Full HD,1920x1080
125,999.0,False,IPS Panel,Full HD,1920x1080
296,599.9,False,None,Full HD,1920x1080
797,597.0,False,None,Full HD,1920x1080
750,1699.0,True,IPS Panel,Full HD,1920x1080


##### `CPU`

In [54]:
def extract_cpu_data(value):
    if "amd" in value.strip().lower():
        # BRAND
        pattern = r"amd"
        is_match = re.search(pattern, value, re.IGNORECASE)
        brand = is_match.group(0) if is_match else None

        # FAMILY
        pattern = r"(?<=AMD\s)(([A-Z])(?:\d{0,2})(-Series)|Ryzen|FX|Athlon|Threadripper|EPYC)"
        is_match = re.search(pattern, value, re.IGNORECASE)

        group1, group2, group3 = is_match.groups()
        is_group1 = group2 is None and group3 is None
        family = group1 if is_group1 else group2 + group3

        # SERIES
        pattern = r"(?P<serie>[AE]\d{1,2})"
        is_match = re.search(pattern, value, re.IGNORECASE)
        series = is_match.group("serie") if is_match else None

        # GEN
        pattern = r"(\d{1})\d{3}[A-Z]?"
        is_match = re.search(pattern, value, re.IGNORECASE)
        generation = is_match.group(1) if is_match else None

        # CLOCK
        pattern = r"(\d[\d\.]*)(?:[A-z]+)$"
        is_match = re.search(pattern, value, re.IGNORECASE)
        clock = is_match.group(1) if is_match else None

        return {
            "cpu_brand": brand,
            "cpu_family": family,
            "cpu_series": series,
            "cpu_gen": generation,
            "cpu_clock": clock
        }

    if "intel" in value.strip().lower():
        # BRAND
        pattern = r"intel"
        is_match = re.search(pattern, value, re.IGNORECASE)
        brand = is_match.group(0) if is_match else None

        # FAMILY
        pattern = r"(?<=Intel)\s(?P<family>Xeon|Atom|Core|Pentium|Celeron)"
        is_match = re.search(pattern, value, re.IGNORECASE)
        family = is_match.group(0) if is_match.group("family") else None

        # SERIES
        pattern = r"((?:Pentium\s|Celeron\s)(Dual|Single|Quad))|((?:Atom|Xeon|Core m?|Atom)\s?([xeim\d]*))"
        is_match = re.search(pattern, value, re.IGNORECASE)
        serie = is_match.group(4) if is_match.group(4) is not None else is_match.group(2) + " Core"

        # GEN
        pattern = r"(\d{1})+\d{3,4}[A-Z]*"
        is_match = re.search(pattern, value, re.IGNORECASE)
        generation = is_match.group(1) if is_match else None

        # CLOCK
        pattern = r"(\d[\d\.]*)(?:[A-z]+)$"
        is_match = re.search(pattern, value, re.IGNORECASE)
        clock = is_match.group(1) if is_match else None


        return {
            "cpu_brand": brand,
            "cpu_family": family,
            "cpu_series": serie,
            "cpu_gen": generation,
            "cpu_clock": clock
        }

cpu_data = pd.DataFrame(df.loc[:, "Cpu"].apply(extract_cpu_data).tolist())
df = pd.concat([df, cpu_data], axis=1)
df.sample(5).iloc[:, -5:]

,cpu_brand,cpu_family,cpu_series,cpu_gen,cpu_clock
29,Intel,Core,i7,7,2.7
744,Intel,Core,i7,8,1.8
702,Intel,Core,i7,7,2.7
341,AMD,A-Series,A9,9,3
387,Intel,Celeron,Dual Core,3,1.6


### `Memory`

In [55]:
def extract_memory_data(value):
    data = {}
    pattern = r"(?P<capacity>[\.\d]*?\d{1,4}(GB|TB))\s(?P<type>Hybrid|SSD|HDD|Flash\sStorage)"

    memories = value.split("+")
    for index, memory in enumerate(memories, 1):
        is_match = re.search(pattern, memory, re.IGNORECASE)
        if is_match:
            data[f"mem_capacity_{index}"] = is_match.group("capacity")
            data[f"mem_type_{index}"] = is_match.group("type")

    columns = ["mem_capacity_1", "mem_type_1", "mem_capacity_2", "mem_type_2"]
    return pd.Series(data)


memory_data = df.loc[:, "Memory"].apply(extract_memory_data)
df = pd.concat([df, memory_data], axis=1)
df.sample(5).iloc[:, -4:]

,mem_capacity_1,mem_type_1,mem_capacity_2,mem_type_2
352,1TB,HDD,NaN,NaN
476,500GB,HDD,NaN,NaN
77,256GB,SSD,NaN,NaN
886,128GB,SSD,1TB,HDD
249,256GB,SSD,NaN,NaN


### `GPU`

In [56]:
def extract_gpu_data(value):
    data = {}
    if "intel" in value.lower():
        pattern = r"(?P<brand>Intel)\s(?P<family>[A-z\s]*)\s(?P<model>\d+[A-z]?)"
        is_match = re.search(pattern, value, re.IGNORECASE)
        if is_match:
            data["gpu_brand"] = is_match.group("brand")
            data["gpu_family"] = is_match.groups("family")
            data["gpu_model"] = is_match.groups("model")

    # return data


mask = df["Gpu"].str.contains("intel", case=False)
df.loc[:, "Gpu"].apply(extract_gpu_data)

,Gpu
0,None
1,None
2,None
3,None
4,None
...,...
907,None
908,None
909,None
910,None


In [57]:
import re
mask = df["Gpu"].str.contains("intel", case=False)
for gpu in df.loc[mask,  "Gpu"]:
    pattern = r"(?P<brand>Intel)\s(?P<family>[A-z\s]*)\s(?P<model>\d+[A-z]?)"
    searchs = re.search(pattern, gpu, re.IGNORECASE)
    print(searchs.groups("Intel"))

('Intel', 'HD Graphics', '520')
('Intel', 'HD Graphics', '6000')
('Intel', 'HD Graphics', '620')
('Intel', 'HD Graphics', '620')
('Intel', 'Iris Graphics', '550')
('Intel', 'HD Graphics', '620')
('Intel', 'HD Graphics', '505')
('Intel', 'HD Graphics', '520')
('Intel', 'UHD Graphics', '620')
('Intel', 'HD Graphics', '405')
('Intel', 'HD Graphics', '520')
('Intel', 'HD Graphics', '620')
('Intel', 'HD Graphics', '400')


AttributeError: 'NoneType' object has no attribute 'groups'

### 2.3 Definir X e y

In [ ]:
X = df.drop(["Price_in_euros"], axis=1)
y = df["Price_in_euros"].copy()

X.shape

In [ ]:
y.shape

### 2.4 Dividir X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
	X,
	y,
	test_size=.20,
	random_state=42
)

In [ ]:
X_train

In [ ]:
y_train

## 3. Procesado de datos

Nuestro target es la columna `Price_in_euros`

In [ ]:
def get_quality(dataframe: pd.DataFrame):
	if dataframe is None:
		raise ValueError("Dataframe sin especificar.")

	if not isinstance(dataframe, pd.DataFrame):
		raise ValueError("El dataframe debe ser pd.DataFrame")

	records = dataframe.shape[0]

	unique = dataframe.nunique().rename("Cardinality")
	cardinality = (unique/records).rename("Uniqueness")
	null = (dataframe.isna().sum()).rename("Null")

	data = pd.DataFrame(data=[unique, cardinality, null])
	return data.T

In [ ]:
get_quality(X_train)

In [ ]:
y_train.describe()

In [ ]:
y_train.isnull().value_counts()

-----------------------------------------------------------------------------------------------------------------

## 4. Modelado

### 4.1 Baseline de modelos

### 4.2 Sacar métricas, valorar los modelos

Recuerda que en la competición se va a evaluar con la métrica de ``RMSE``.

### 4.3 Optimización (up to you 🫰🏻)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir ``test.csv``

**RECUERDA: APLICAR LAS TRANSFORMACIONES QUE HAYAS REALIZADO EN `train.csv` a `test.csv`.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

## 1. Carga los datos de `test.csv` para predecir.


In [ ]:
X_pred = pd.read_csv("./data/test.csv")
X_pred.head()

In [ ]:
X_pred.tail()

In [ ]:
X_pred.info()

 ## 2. Replicar el procesado para ``test.csv``

In [ ]:
X_pred

In [ ]:
predictions_submit = model.predict(X_pred)
predictions_submit

**¡OJO! ¿Por qué me da error?**

IMPORTANTE:

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO **BORRAR FILAS**, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER **391 FILAS**, SI O SI

**Entonces, si al cargar los datos de ``train.csv`` usaste `index_col=0`, ¿tendré que hacer lo también para el `test.csv`?**

In [ ]:
# ¿Qué opináis?
# ¿Sí, no?

![wow.jpeg](attachment:wow.jpeg)

## 3. **¿Qué es lo que subirás a Kaggle?**

**Para subir a Kaggle la predicción esta tendrá que tener una forma específica.**

En este caso, la **MISMA** forma que `sample_submission.csv`.

In [ ]:
sample = pd.read_csv("data/sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
sample.shape

## 4. Mete tus predicciones en un dataframe llamado ``submission``.

In [ ]:
#¿Cómo creamos la submission?
submission = pd.DataFrame()

In [ ]:
submission.head()

In [ ]:
submission.shape

## 5. Pásale el CHEQUEADOR para comprobar que efectivamente está listo para subir a Kaggle.

In [ ]:
def chequeador(df_to_submit):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.

    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.

    Si no, LEE EL MENSAJE Y HAZLE CASO.

    Si aún no:
    - apaga tu ordenador,
    - date una vuelta,
    - enciendelo otra vez,
    - abre este notebook y
    - leelo todo de nuevo.
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit.laptop_ID.all() == sample.laptop_ID.all():
                print("You're ready to submit!")
                df_to_submit.to_csv("submission.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://www.mihaileric.com/static/evaluation-meme-e0a350f278a36346e6d46b139b1d0da0-ed51e.jpg", "gfg.png")
                img = Image.open("gfg.png")
                img.show()
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto del TA: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `test.csv`. Lloro.")

In [ ]:
chequeador(submission)